In [4]:
from bs4 import BeautifulSoup
import requests
import urllib
import ssl
import pandas as pd

In [5]:
url = "http://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp"

context = ssl._create_unverified_context()
req = urllib.request.urlopen(url, context=context)
html = req.read()
html

b'\r\n<!doctype html>\r\n<html class="no-js" lang="pt-br">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1" />\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0" />\r\n<link rel="stylesheet" href="css/foundation.css" />\n<link rel="stylesheet" href="css/jquery.ui.datepicker.css" />\n<script src="js/vendor/modernizr.js"></script>\r\n\r\n<script type="text/javascript">\r\nfunction Retroativo(theForm) {\r\n    if(!CkDate(theForm.dData1, theForm.dData1.value))\r\n        {\r\n            \r\n                alert("Selecione o per\xedodo que deseja fazer a consulta.");\r\n            \r\n            theForm.dData1.focus();\r\n            theForm.dData1.select();\r\n            return false;\r\n        }\r\n    if(!CkDate(theForm.dData2, theForm.dData2.value))\r\n        {\r\n            \r\n                alert("Selecione o per\xedodo que deseja fazer a consulta.");\r\n            \r\n            theForm.dData2.focus();\r\n         

In [67]:
soup = BeautifulSoup(html, "html5lib")

In [68]:
print(soup.find_all("table")[0].prettify())

<table id="tb_principal1">
 <thead>
  <tr>
   <th class="text-center" rowspan="2">
    Dias Corridos
   </th>
   <th class="text-center" colspan="2">
    DI x pré
   </th>
  </tr>
  <tr>
   <th class="text-center" width="%">
    252
    <sup>
     (2)(4)
    </sup>
   </th>
   <th class="text-center" width="%">
    360
    <sup>
     (1)
    </sup>
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td>
    1
   </td>
   <td class="text-right">
    7,65
   </td>
   <td class="text-right">
    0,00
   </td>
  </tr>
  <tr>
   <td>
    2
   </td>
   <td class="text-right">
    8,42
   </td>
   <td class="text-right">
    12,23
   </td>
  </tr>
  <tr>
   <td>
    7
   </td>
   <td class="text-right">
    8,88
   </td>
   <td class="text-right">
    9,07
   </td>
  </tr>
  <tr>
   <td>
    8
   </td>
   <td class="text-right">
    8,93
   </td>
   <td class="text-right">
    9,60
   </td>
  </tr>
  <tr>
   <td>
    9
   </td>
   <td class="text-right">
    8,97
   </td>
   <td class="text-right"

In [118]:
soup.find_all("table")[0].find_all('tr')[0]

<tr>
<th class="text-center" rowspan="2">Dias Corridos</th>

<th class="text-center" colspan="2">
DI x pré
</th>


</tr>

In [94]:
html_tabela = soup.find_all("table")[0]

In [119]:
trs = html_tabela.find_all('tr')

In [124]:
trs[0]

<tr>
<th class="text-center" rowspan="2">Dias Corridos</th>

<th class="text-center" colspan="2">
DI x pré
</th>


</tr>

In [128]:
html_tabela

<table id="tb_principal1">


<thead>
<tr>
<th class="text-center" rowspan="2">Dias Corridos</th>

<th class="text-center" colspan="2">
DI x pré
</th>


</tr>

<tr><th class="text-center" width="%">
252<sup>(2)(4)</sup>
</th>

<th class="text-center" width="%">
360<sup>(1)</sup>
</th>

</tr>


</thead>
<tbody><tr><td>1</td>

<td class="text-right">7,65</td>

<td class="text-right">0,00</td>

</tr>



<tr><td>2</td>

<td class="text-right">8,42</td>

<td class="text-right">12,23</td>

</tr>



<tr><td>7</td>

<td class="text-right">8,88</td>

<td class="text-right">9,07</td>

</tr>



<tr><td>8</td>

<td class="text-right">8,93</td>

<td class="text-right">9,60</td>

</tr>



<tr><td>9</td>

<td class="text-right">8,97</td>

<td class="text-right">10,01</td>

</tr>



<tr><td>13</td>

<td class="text-right">9,01</td>

<td class="text-right">8,91</td>

</tr>



<tr><td>14</td>

<td class="text-right">9,03</td>

<td class="text-right">9,22</td>

</tr>



<tr><td>21</td>

<td class="text-ri

In [130]:
def ConverteTabelaHTML(tabela): # converte tabela HTML bs4 em numérico       
    """
    Converte uma tag <table> do HTML (sendo objeto oriundo do BeautifulSoup) em uma matriz que pode ser convertida em DataFrame.
    A função separa o HTML de cada linha <tr> em uma lista, e depois separa os dígitos contidos em cada linha <td> em outra lista.
    Ela também separa a primeira linha <tr> e os dígitos contidos nela <th> como cabeçalho.
    """
    
    rows = []
    table_rows = tabela.find_all('tr') # separa o HTML de cada linha (table row) numa lista
    
    header_row = [td.get_text(strip = True) for td in table_rows[0].find_all('th')] # primeira linha deve conter tags <th> (table headers) com tags <td> dentro (table digits)
    
    if header_row: # retorna True se a lista não está vazia
        rows.append(header_row) # primeira linha é o cabeçalho, se ele existir
        table_rows = table_rows[1:]
        
    for row in table_rows: # para cada linha da lista
        rows.append([num.get_text(strip=True) for num in row.find_all('td')]) # separa o valor contido nas tags <td> (table digits)
    
    return rows

tabela = ConverteTabelaHTML(html_tabela)
tabela[:10]

[['Dias Corridos', 'DI x pré'],
 [],
 ['1', '7,65', '0,00'],
 ['2', '8,42', '12,23'],
 ['7', '8,88', '9,07'],
 ['8', '8,93', '9,60'],
 ['9', '8,97', '10,01'],
 ['13', '9,01', '8,91'],
 ['14', '9,03', '9,22'],
 ['21', '9,08', '9,28']]

In [133]:
tabela.pop(1)
df = pd.DataFrame(tabela)
df.head(15)

,0,1,2
0,Dias Corridos,DI x pré,None
1,7,"8,88","9,07"
2,8,"8,93","9,60"
3,9,"8,97","10,01"
4,13,"9,01","8,91"
5,14,"9,03","9,22"
6,21,"9,08","9,28"
7,22,"9,09","9,46"
8,26,"9,10","9,00"
9,30,"9,12","9,57"
